# What is the impact of class imbalance and different sampling methods on the performance of machine learning algorithms for classification?

Using a (synthetic) imbalanced dataset (hier nog welke), we will compare the performance of 2 classification algorithms. We will then investigate the impact of class imbalance on the performance of these algoritms, using different sampling methods such as random under-sampling, random over-sampling and SMOTE. Specifically, we will measure and compare the in-sample and out-of-sample performance of the algorithms under varying degrees of class imbalance and sampling methods.

# Random under-sampling & random over-sampling

The most naive class of techniques for dealing with class imbalances is sampling. Random under-sampling involves removing some of the majority class samples at random until the dataset is balanced. Whereas random over-sampling involves duplicating some of the minority class samples at random until the dataset is balanced.

# SMOTE

SMOTE is a more sophisticated technique that generates synthetic samples for the minority class based on the characteristics of that class. This helps to balance the classes while preserving the information in the dataset.